# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import time
from PIL import Image
import numpy as np


import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from workspace_utils import active_session, keep_awake

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(224),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

data_transforms = transforms.Compose([transforms.Resize(224),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], 
                                                      [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms)
test_data = datasets.ImageFolder(test_dir, transform=data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
# TODO: Build and train your network
# Model definition
model = models.densenet121(pretrained = True)

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 70086559.98it/s]


In [6]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [7]:
# Define classifier
model.classifier = nn.Sequential(nn.Linear(1024, 1000),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(1000, 102),
                                 nn.LogSoftmax(dim=1))

In [8]:
# Define Criterion and Optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.002)

In [9]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [10]:
# Run and Train the model

with active_session():
    
    epochs = 3
    steps = 0
    running_loss = 0
    for epoch in range(epochs):
        Time = time.time()
        for inputs, labels in train_loader:
            model.train()
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in valid_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)

                    test_loss += batch_loss.item()

                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Step {steps}.. "
                  f"Time: {round(time.time() - Time,2)} seconds.. "
                  f"Train loss: {running_loss/len(train_loader):.3f}.. "
                  f"Test loss: {test_loss/len(valid_loader):.3f}.. "
                  f"Test accuracy: {round((accuracy.item()/len(valid_loader))*100,2):.3f} %.. ") 
            running_loss = 0

Epoch 1/3.. Step 1.. Time: 15.41 seconds.. Train loss: 0.045.. Test loss: 4.610.. Test accuracy: 3.710 %.. 
Epoch 1/3.. Step 2.. Time: 28.85 seconds.. Train loss: 0.045.. Test loss: 4.622.. Test accuracy: 3.970 %.. 
Epoch 1/3.. Step 3.. Time: 42.24 seconds.. Train loss: 0.046.. Test loss: 4.612.. Test accuracy: 4.090 %.. 
Epoch 1/3.. Step 4.. Time: 55.73 seconds.. Train loss: 0.045.. Test loss: 4.637.. Test accuracy: 5.650 %.. 
Epoch 1/3.. Step 5.. Time: 69.33 seconds.. Train loss: 0.046.. Test loss: 4.623.. Test accuracy: 3.850 %.. 
Epoch 1/3.. Step 6.. Time: 82.71 seconds.. Train loss: 0.048.. Test loss: 4.574.. Test accuracy: 4.690 %.. 
Epoch 1/3.. Step 7.. Time: 96.08 seconds.. Train loss: 0.045.. Test loss: 4.504.. Test accuracy: 3.000 %.. 
Epoch 1/3.. Step 8.. Time: 109.49 seconds.. Train loss: 0.044.. Test loss: 4.438.. Test accuracy: 4.930 %.. 
Epoch 1/3.. Step 9.. Time: 122.99 seconds.. Train loss: 0.042.. Test loss: 4.375.. Test accuracy: 6.610 %.. 
Epoch 1/3.. Step 10.. Time

Epoch 1/3.. Step 76.. Time: 1011.43 seconds.. Train loss: 0.020.. Test loss: 1.607.. Test accuracy: 63.700 %.. 
Epoch 1/3.. Step 77.. Time: 1024.62 seconds.. Train loss: 0.021.. Test loss: 1.576.. Test accuracy: 65.370 %.. 
Epoch 1/3.. Step 78.. Time: 1037.76 seconds.. Train loss: 0.022.. Test loss: 1.547.. Test accuracy: 64.300 %.. 
Epoch 1/3.. Step 79.. Time: 1050.97 seconds.. Train loss: 0.020.. Test loss: 1.539.. Test accuracy: 61.320 %.. 
Epoch 1/3.. Step 80.. Time: 1064.25 seconds.. Train loss: 0.022.. Test loss: 1.529.. Test accuracy: 61.100 %.. 
Epoch 1/3.. Step 81.. Time: 1077.38 seconds.. Train loss: 0.023.. Test loss: 1.505.. Test accuracy: 61.940 %.. 
Epoch 1/3.. Step 82.. Time: 1090.48 seconds.. Train loss: 0.017.. Test loss: 1.473.. Test accuracy: 64.120 %.. 
Epoch 1/3.. Step 83.. Time: 1103.62 seconds.. Train loss: 0.022.. Test loss: 1.456.. Test accuracy: 64.420 %.. 
Epoch 1/3.. Step 84.. Time: 1116.84 seconds.. Train loss: 0.018.. Test loss: 1.454.. Test accuracy: 65.3

Epoch 2/3.. Step 150.. Time: 620.72 seconds.. Train loss: 0.013.. Test loss: 0.921.. Test accuracy: 79.380 %.. 
Epoch 2/3.. Step 151.. Time: 633.78 seconds.. Train loss: 0.013.. Test loss: 0.900.. Test accuracy: 78.720 %.. 
Epoch 2/3.. Step 152.. Time: 646.94 seconds.. Train loss: 0.015.. Test loss: 0.895.. Test accuracy: 77.400 %.. 
Epoch 2/3.. Step 153.. Time: 660.1 seconds.. Train loss: 0.015.. Test loss: 0.883.. Test accuracy: 78.050 %.. 
Epoch 2/3.. Step 154.. Time: 673.24 seconds.. Train loss: 0.016.. Test loss: 0.885.. Test accuracy: 77.330 %.. 
Epoch 2/3.. Step 155.. Time: 686.44 seconds.. Train loss: 0.014.. Test loss: 0.891.. Test accuracy: 75.940 %.. 
Epoch 2/3.. Step 156.. Time: 699.58 seconds.. Train loss: 0.014.. Test loss: 0.887.. Test accuracy: 76.540 %.. 
Epoch 2/3.. Step 157.. Time: 712.61 seconds.. Train loss: 0.016.. Test loss: 0.885.. Test accuracy: 76.280 %.. 
Epoch 2/3.. Step 158.. Time: 725.7 seconds.. Train loss: 0.012.. Test loss: 0.883.. Test accuracy: 77.040

Epoch 3/3.. Step 224.. Time: 235.74 seconds.. Train loss: 0.011.. Test loss: 0.651.. Test accuracy: 83.660 %.. 
Epoch 3/3.. Step 225.. Time: 248.7 seconds.. Train loss: 0.010.. Test loss: 0.645.. Test accuracy: 84.500 %.. 
Epoch 3/3.. Step 226.. Time: 261.59 seconds.. Train loss: 0.012.. Test loss: 0.638.. Test accuracy: 85.170 %.. 
Epoch 3/3.. Step 227.. Time: 274.53 seconds.. Train loss: 0.010.. Test loss: 0.639.. Test accuracy: 84.990 %.. 
Epoch 3/3.. Step 228.. Time: 287.5 seconds.. Train loss: 0.011.. Test loss: 0.635.. Test accuracy: 85.210 %.. 
Epoch 3/3.. Step 229.. Time: 300.43 seconds.. Train loss: 0.014.. Test loss: 0.629.. Test accuracy: 85.790 %.. 
Epoch 3/3.. Step 230.. Time: 313.36 seconds.. Train loss: 0.015.. Test loss: 0.621.. Test accuracy: 85.340 %.. 
Epoch 3/3.. Step 231.. Time: 326.31 seconds.. Train loss: 0.015.. Test loss: 0.618.. Test accuracy: 85.270 %.. 
Epoch 3/3.. Step 232.. Time: 339.16 seconds.. Train loss: 0.011.. Test loss: 0.626.. Test accuracy: 84.850

Epoch 3/3.. Step 298.. Time: 1201.48 seconds.. Train loss: 0.011.. Test loss: 0.550.. Test accuracy: 86.130 %.. 
Epoch 3/3.. Step 299.. Time: 1214.53 seconds.. Train loss: 0.013.. Test loss: 0.543.. Test accuracy: 86.610 %.. 
Epoch 3/3.. Step 300.. Time: 1227.56 seconds.. Train loss: 0.008.. Test loss: 0.543.. Test accuracy: 85.740 %.. 
Epoch 3/3.. Step 301.. Time: 1240.6 seconds.. Train loss: 0.009.. Test loss: 0.546.. Test accuracy: 86.100 %.. 
Epoch 3/3.. Step 302.. Time: 1253.68 seconds.. Train loss: 0.009.. Test loss: 0.548.. Test accuracy: 85.890 %.. 
Epoch 3/3.. Step 303.. Time: 1266.76 seconds.. Train loss: 0.009.. Test loss: 0.556.. Test accuracy: 85.440 %.. 
Epoch 3/3.. Step 304.. Time: 1279.91 seconds.. Train loss: 0.013.. Test loss: 0.563.. Test accuracy: 84.860 %.. 
Epoch 3/3.. Step 305.. Time: 1293.03 seconds.. Train loss: 0.008.. Test loss: 0.571.. Test accuracy: 84.640 %.. 
Epoch 3/3.. Step 306.. Time: 1306.2 seconds.. Train loss: 0.012.. Test loss: 0.577.. Test accurac

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [12]:
# TODO: Do validation on the test set
with active_session():

    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        Time = time.time()
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))

        print(f"Time: {round(time.time() - Time,2)} seconds.. "
              f"Test loss: {test_loss/len(valid_loader):.3f}.. "
              f"Test accuracy: {round((accuracy.item()/len(test_loader))*100,2):.3f} %.. ")

Time: 12.21 seconds.. Test loss: 0.607.. Test accuracy: 82.360 %.. 


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [13]:
# TODO: Save the checkpoint .

checkpoint_dir = 'CheckPoint/'
checkpoint_file = 'CheckFile_3.pth'

torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_dir + checkpoint_file)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [10]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

checkpoint_dir = 'CheckPoint/'
checkpoint_file = 'CheckFile_3.pth'

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
 
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epochs = checkpoint['epoch']
        
    model.eval()
        
    return model


model = load_checkpoint(checkpoint_dir + checkpoint_file)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [49]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

    img = Image.open(image)
    
    if img.size[0] > img.size[1]:
        img.thumbnail((10000, 256))
    else:
        img.thumbnail((256, 10000))
     
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin,   
                      top_margin))
    
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (img - mean)/std
    
    img = img.transpose((2, 0, 1))
    
    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [50]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

                  
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [51]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''  
    # TODO: Implement the code to predict the class from an image file
    
    img = process_image(image_path)
    model.eval()
    with torch.no_grad():
        inputs = torch.from_numpy(img).type(torch.FloatTensor)
        inputs.unsqueeze_(0)
        
        logps = model.forward(inputs)
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(5)
    return probs, classes

In [52]:
data_dir = 'flowers'
test_dir = data_dir + '/test/102/'
file_name = 'image_08004.jpg'

model = load_checkpoint(checkpoint_dir + checkpoint_file)
image_path = test_dir + file_name

predict(image_path, model, topk=5)

print(probs)
print(classes)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.DoubleTensor for argument #2 'weight'

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes